In [1]:
import pandas as pd
import io
import requests
import glob
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()
from bs4 import BeautifulSoup
import re
import calendar
import locale
import yfinance
# Set the locale to German
import pygsheets
import datawrapper
from datetime import datetime
from anthropic import Anthropic

import locale

# Set the locale to English (United States)
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:

def load_excel_from_url(url):
    """Loads an Excel file from a URL into a pandas DataFrame.

    Args:
        url: The URL of the Excel file.

    Returns:
        A pandas DataFrame containing the data from the Excel file, or None if
        an error occurs.
    """
    try:
        headers = {
               "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
           }
        response = requests.get(url, headers=headers, stream=True) # Make the request with headers
        response.raise_for_status()  # Raise an exception for bad status codes (like 403)

        # Read the content into a BytesIO object for pandas
        excel_data = io.BytesIO(response.content)

        return excel_data
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None


In [3]:
us_trade="https://www.census.gov/foreign-trade/statistics/country/index.html#:~:text=Download%20Full%20Dataset%20(XLSX%20%2D%200.1%20MB)"

us_stats=load_excel_from_url(us_trade)
us_stats

In [4]:
url="https://www.census.gov/foreign-trade/statistics/country/ctyseasonal.xlsx"


In [5]:

# Download the file using requests
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BytesIO object from the response content
    excel_file = io.BytesIO(response.content)
    
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file, engine='openpyxl')  # Specify engine if needed
    print(df)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

     year  cty_code               cty_desc          BJAN          BFEB  \
0    2009        20    All other countries -7.688157e+09 -6.061251e+09   
1    2009      3510                 Brazil  5.109941e+08  4.469206e+08   
2    2009        17               CAFTA-DR  3.034300e+08  2.818892e+08   
3    2009      1220                 Canada -1.790673e+09 -1.893448e+09   
4    2009      5700                  China -2.036187e+10 -1.900556e+10   
..    ...       ...                    ...           ...           ...   
389  2025         9  South/Central America  4.319924e+09  4.621780e+09   
390  2025      4419            Switzerland -2.299469e+10 -2.001357e+10   
391  2025      5830                 Taiwan -7.480248e+09 -8.955526e+09   
392  2025      4120         United Kingdom -6.731581e+08  3.245911e+09   
393  2025      5520                Vietnam -1.229118e+10 -1.249563e+10   

             BMAR          BAPR          BMAY          BJUN          BJUL  \
0   -6.221323e+09 -7.484819e+09 -5

In [6]:
df.columns

Index(['year', 'cty_code', 'cty_desc', 'BJAN', 'BFEB', 'BMAR', 'BAPR', 'BMAY',
       'BJUN', 'BJUL', 'BAUG', 'BSEP', 'BOCT', 'BNOV', 'BDEC', 'BQ1', 'BQ2',
       'BQ3', 'BQ4', 'EJAN', 'EFEB', 'EMAR', 'EAPR', 'EMAY', 'EJUN', 'EJUL',
       'EAUG', 'ESEP', 'EOCT', 'ENOV', 'EDEC', 'EQ1', 'EQ2', 'EQ3', 'EQ4',
       'IJAN', 'IFEB', 'IMAR', 'IAPR', 'IMAY', 'IJUN', 'IJUL', 'IAUG', 'ISEP',
       'IOCT', 'INOV', 'IDEC', 'IQ1', 'IQ2', 'IQ3', 'IQ4'],
      dtype='object')

In [7]:
df.cty_desc.unique()

array(['All other countries', 'Brazil', 'CAFTA-DR', 'Canada', 'China',
       'European Union', 'France', 'Germany', 'India', 'Italy', 'Japan',
       'Korea, South', 'Mexico', 'Newly Industrialized Countries', 'OPEC',
       'Saudi Arabia', 'South/Central America', 'United Kingdom',
       'Hong Kong', 'Singapore', 'Taiwan', 'Australia', 'Belgium',
       'Ireland', 'Israel', 'Malaysia', 'Netherlands', 'Switzerland',
       'Vietnam'], dtype=object)

In [8]:
base_col=["year","cty_code","cty_desc"]
export_cols=[]
import_cols=[]
for col in df.columns:
    if col[0]=="E":
        export_cols.append(col)
    elif col[0]=="I":
        import_cols.append(col)

base_export_cols=base_col+export_cols
base_import_cols=base_col+import_cols
base_import_cols
export_df=df[base_export_cols]
export_df

,year,cty_code,cty_desc,EJAN,EFEB,EMAR,EAPR,EMAY,EJUN,EJUL,EAUG,ESEP,EOCT,ENOV,EDEC,EQ1,EQ2,EQ3,EQ4
0,2009,20,All other countries,3.092714e+10,3.222777e+10,3.163669e+10,2.979531e+10,3.143728e+10,3.199560e+10,3.202405e+10,3.228728e+10,3.320518e+10,3.501005e+10,3.566379e+10,3.594004e+10,9.479160e+10,9.322820e+10,9.751652e+10,1.066139e+11
1,2009,3510,Brazil,2.310141e+09,2.109861e+09,1.955903e+09,1.938554e+09,2.024336e+09,2.075956e+09,2.122650e+09,2.094565e+09,2.247110e+09,2.364203e+09,2.303859e+09,2.548317e+09,6.375905e+09,6.038846e+09,6.464325e+09,7.216379e+09
2,2009,17,CAFTA-DR,1.594814e+09,1.655722e+09,1.585226e+09,1.413914e+09,1.544395e+09,1.701161e+09,1.627044e+09,1.669154e+09,1.683253e+09,1.745002e+09,1.868941e+09,1.855674e+09,4.835761e+09,4.659470e+09,4.979451e+09,5.469618e+09
3,2009,1220,Canada,1.607276e+10,1.658391e+10,1.551450e+10,1.558492e+10,1.555190e+10,1.559846e+10,1.750951e+10,1.793495e+10,1.800158e+10,1.796037e+10,1.903583e+10,1.930928e+10,4.817117e+10,4.673528e+10,5.344604e+10,5.630547e+10
4,2009,5700,China,4.529859e+09,4.900156e+09,5.187716e+09,5.489906e+09,5.423565e+09,5.808423e+09,5.465313e+09,5.697671e+09,6.215815e+09,6.313081e+09,6.972480e+09,7.492694e+09,1.461773e+10,1.672189e+10,1.737880e+10,2.077826e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2025,9,South/Central America,1.799277e+10,1.847082e+10,1.794769e+10,1.789054e+10,1.811507e+10,1.790981e+10,1.817776e+10,NaN,NaN,NaN,NaN,NaN,5.441128e+10,5.391542e+10,NaN,NaN
390,2025,4419,Switzerland,1.854150e+09,2.509547e+09,3.539151e+09,9.118536e+09,6.480149e+09,4.076431e+09,2.888445e+09,NaN,NaN,NaN,NaN,NaN,7.902847e+09,1.967512e+10,NaN,NaN
391,2025,5830,Taiwan,3.698952e+09,4.101151e+09,4.286813e+09,4.974215e+09,4.645937e+09,4.195714e+09,4.757503e+09,NaN,NaN,NaN,NaN,NaN,1.208692e+10,1.381587e+10,NaN,NaN
392,2025,4120,United Kingdom,5.982091e+09,9.365912e+09,8.244228e+09,8.996384e+09,7.816502e+09,7.329189e+09,7.296015e+09,NaN,NaN,NaN,NaN,NaN,2.359223e+10,2.414208e+10,NaN,NaN


In [9]:
import_df=df[base_import_cols]


In [10]:
def df_for_country(cols,country):    
    df_country=df[df["cty_desc"]==country]
    filtered_entries = [entry for entry in cols if 'Q' not in entry ]
    df_long_country=pd.melt(df_country, id_vars=['year'], value_vars=filtered_entries)
    df_long_country["month"]=df_long_country.variable.str[1:]
    df_long_country["date"]=df_long_country["month"]+"-"+df_long_country["year"].astype(str)
    df_long_country["date"] = pd.to_datetime(df_long_country["date"], format="%b-%Y")
    df_long_country.sort_values(by="date", ascending=True, inplace=True)
    df_long_red=df_long_country[["date","value"]]
    

    return(df_long_red)

In [11]:
def comp_calc(country):
    locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')  # For Linux/Mac

    df_sorted_import=df_for_country(import_cols,country)
    df_sorted_export=df_for_country(export_cols,country)
    df_sorted_import_export=pd.merge(df_sorted_export,df_sorted_import, on="date", how="outer",suffixes=("_export", "_import"))
    df_sorted_import_export["diff"]=df_sorted_import_export["value_export"]-df_sorted_import_export["value_import"]
    last_years=df_sorted_import_export[df_sorted_import_export["date"]>"2021-01-01"]
    last_years=last_years.dropna().reset_index(drop=True)
    last_years
    comparison_df=last_years[(last_years["date"]>"01-01-2021")&(last_years["date"]<"01-01-2025")]
    under_trump=last_years[(last_years["date"]>="01-01-2025")]
    array=[]
    for index, row in under_trump.iloc[-6:].iterrows():
        print(f"Date: {row['date']}, Export: {row['value_export']}, Import: {row['value_import']}, Import: {row['diff']}")
        filtered_df = comparison_df[(comparison_df['date'].dt.month == row['date'].month) & (comparison_df['date'].dt.day == row['date'].day)]
        locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')

        month_name = calendar.month_name[row['date'].month]

        inner_array=[month_name]
        for col in filtered_df.columns[1:]:

            value_mean=filtered_df[col].mean()
            relative_perc=row[col]/value_mean
            diff_perc_=(relative_perc-1)*100
            inner_array.append(diff_perc_)

  

        array.append(inner_array)
    comp_df=pd.DataFrame(array,columns=["date","relative_export","relative_import","relative_diff"])
    comp_df=comp_df.T
    comp_df.columns=comp_df.iloc[0]
    comp_df=comp_df.iloc[1:-1]
    return last_years, comp_df


In [12]:
df_results={}
for country in df.cty_desc.unique():
    print(country)
    comp_objects=comp_calc(country)
    df_results[country]=comp_objects[1]
df

All other countries
Date: 2025-02-01 00:00:00, Export: 33573727186.71178, Import: 47534159304.165855, Import: -13960432117.454075
Date: 2025-03-01 00:00:00, Export: 34203178188.371456, Import: 50206383894.7326, Import: -16003205706.361141
Date: 2025-04-01 00:00:00, Export: 34348754124.168625, Import: 45306873476.503845, Import: -10958119352.33522
Date: 2025-05-01 00:00:00, Export: 33841546566.304478, Import: 44319861346.34381, Import: -10478314780.039333
Date: 2025-06-01 00:00:00, Export: 34707383167.50712, Import: 45723395668.826866, Import: -11016012501.319748
Date: 2025-07-01 00:00:00, Export: 35470349437.2231, Import: 45002419480.968414, Import: -9532070043.745316
Brazil
Date: 2025-02-01 00:00:00, Export: 4518647354.618509, Import: 4114285534.6831856, Import: 404361819.9353237
Date: 2025-03-01 00:00:00, Export: 4604176196.164625, Import: 4165765352.0453496, Import: 438410844.11927557
Date: 2025-04-01 00:00:00, Export: 4475907636.662008, Import: 3451813444.4628577, Import: 102409419

,year,cty_code,cty_desc,BJAN,BFEB,BMAR,BAPR,BMAY,BJUN,BJUL,...,IJUL,IAUG,ISEP,IOCT,INOV,IDEC,IQ1,IQ2,IQ3,IQ4
0,2009,20,All other countries,-7.688157e+09,-6.061251e+09,-6.221323e+09,-7.484819e+09,-5.537787e+09,-6.201629e+09,-8.128920e+09,...,4.015297e+10,3.968960e+10,4.240630e+10,4.296826e+10,4.423691e+10,4.572743e+10,1.147623e+11,1.124524e+11,1.222489e+11,1.329326e+11
1,2009,3510,Brazil,5.109941e+08,4.469206e+08,3.447423e+08,5.177740e+08,4.190184e+08,4.636909e+08,4.404457e+08,...,1.682204e+09,1.613042e+09,1.690305e+09,1.610670e+09,1.983716e+09,1.778059e+09,5.073248e+09,4.638363e+09,4.985551e+09,5.372444e+09
2,2009,17,CAFTA-DR,3.034300e+08,2.818892e+08,2.177867e+08,6.088876e+07,1.325944e+08,1.921538e+08,-4.636460e+07,...,1.673409e+09,1.675474e+09,1.685764e+09,1.791865e+09,1.882657e+09,1.826492e+09,4.032656e+09,4.273833e+09,5.034646e+09,5.501014e+09
3,2009,1220,Canada,-1.790673e+09,-1.893448e+09,-1.384375e+09,-1.091626e+09,-8.162940e+08,-1.853868e+09,-2.322835e+09,...,1.983234e+10,1.945058e+10,1.978832e+10,2.060733e+10,2.105888e+10,2.177427e+10,5.323966e+10,5.049706e+10,5.907124e+10,6.344048e+10
4,2009,5700,China,-2.036187e+10,-1.900556e+10,-2.037261e+10,-1.842612e+10,-1.840727e+10,-1.757020e+10,-1.826092e+10,...,2.372623e+10,2.377449e+10,2.462898e+10,2.558478e+10,2.595788e+10,2.721828e+10,7.435777e+10,7.112548e+10,7.212970e+10,7.876093e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2025,9,South/Central America,4.319924e+09,4.621780e+09,2.991398e+09,3.294295e+09,3.347217e+09,4.355232e+09,4.596051e+09,...,1.358171e+10,NaN,NaN,NaN,NaN,NaN,4.247818e+10,4.291868e+10,NaN,NaN
390,2025,4419,Switzerland,-2.299469e+10,-2.001357e+10,-1.540641e+10,3.538335e+09,3.288457e+09,-3.482191e+07,-7.675648e+09,...,1.056409e+10,NaN,NaN,NaN,NaN,NaN,6.631752e+10,1.288315e+10,NaN,NaN
391,2025,5830,Taiwan,-7.480248e+09,-8.955526e+09,-8.915797e+09,-9.665720e+09,-1.149461e+10,-1.287696e+10,-1.346071e+10,...,1.821821e+10,NaN,NaN,NaN,NaN,NaN,3.743849e+10,4.785316e+10,NaN,NaN
392,2025,4120,United Kingdom,-6.731581e+08,3.245911e+09,1.443552e+09,4.347961e+09,3.015399e+09,2.230419e+09,1.767927e+09,...,5.528088e+09,NaN,NaN,NaN,NaN,NaN,1.957593e+10,1.454830e+10,NaN,NaN


In [13]:
overall_df=pd.DataFrame()
for country in df_results.keys():
    df_single=df_results[country]
    df_single["country"]=country
    df_single["category"]=df_single.index
    overall_df=pd.concat([overall_df, df_single], ignore_index=True)
overall_df

date,Februar,März,April,Mai,Juni,Juli,country,category
0,14.348629,12.502712,12.127676,11.43591,15.620113,14.914017,All other countries,relative_export
1,22.53502,27.170855,13.711973,9.668469,13.609099,11.4489,All other countries,relative_import
2,16.579207,15.973162,7.646772,15.495532,12.421021,2.627113,Brazil,relative_export
3,38.117221,39.167065,13.195269,39.003309,8.188454,17.244549,Brazil,relative_import
4,7.066363,8.650665,4.157202,3.304412,1.310821,17.510049,CAFTA-DR,relative_export
5,13.745174,27.561837,22.096923,13.751477,19.677708,17.209726,CAFTA-DR,relative_import
6,10.418587,6.249624,-5.156551,-4.955268,-3.681084,-8.782964,Canada,relative_export
7,16.581521,4.599363,-13.988413,-13.826042,-15.412454,-7.448395,Canada,relative_import
8,-8.231457,-14.660652,-31.846129,-42.624377,-14.750128,-16.909466,China,relative_export
9,-5.731318,-19.445894,-30.277664,-45.710313,-50.884338,-36.682457,China,relative_import


In [14]:
overall_df["min"]=overall_df[overall_df.columns[0:5]].min(axis=1)
overall_df

date,Februar,März,April,Mai,Juni,Juli,country,category,min
0,14.348629,12.502712,12.127676,11.43591,15.620113,14.914017,All other countries,relative_export,11.43591
1,22.53502,27.170855,13.711973,9.668469,13.609099,11.4489,All other countries,relative_import,9.668469
2,16.579207,15.973162,7.646772,15.495532,12.421021,2.627113,Brazil,relative_export,7.646772
3,38.117221,39.167065,13.195269,39.003309,8.188454,17.244549,Brazil,relative_import,8.188454
4,7.066363,8.650665,4.157202,3.304412,1.310821,17.510049,CAFTA-DR,relative_export,1.310821
5,13.745174,27.561837,22.096923,13.751477,19.677708,17.209726,CAFTA-DR,relative_import,13.745174
6,10.418587,6.249624,-5.156551,-4.955268,-3.681084,-8.782964,Canada,relative_export,-5.156551
7,16.581521,4.599363,-13.988413,-13.826042,-15.412454,-7.448395,Canada,relative_import,-15.412454
8,-8.231457,-14.660652,-31.846129,-42.624377,-14.750128,-16.909466,China,relative_export,-42.624377
9,-5.731318,-19.445894,-30.277664,-45.710313,-50.884338,-36.682457,China,relative_import,-50.884338


In [15]:
overall_df.sort_values(by="min", ascending=True, inplace=True)
overall_df

date,Februar,März,April,Mai,Juni,Juli,country,category,min
37,-18.048016,-52.632136,-52.973824,-42.805113,-5.130823,-14.56372,Hong Kong,relative_import,-52.973824
31,-17.895517,-42.614814,-52.209509,-42.341363,-38.482087,-48.593953,Saudi Arabia,relative_import,-52.209509
9,-5.731318,-19.445894,-30.277664,-45.710313,-50.884338,-36.682457,China,relative_import,-50.884338
8,-8.231457,-14.660652,-31.846129,-42.624377,-14.750128,-16.909466,China,relative_export,-42.624377
49,16.634459,35.453451,17.855324,-30.483759,-32.04145,-7.307299,Israel,relative_import,-32.04145
55,383.650769,212.454089,24.348699,-30.764951,-16.559206,134.000637,Switzerland,relative_import,-30.764951
53,-10.051591,29.390213,12.854389,30.232005,-16.972966,10.259109,Netherlands,relative_import,-16.972966
7,16.581521,4.599363,-13.988413,-13.826042,-15.412454,-7.448395,Canada,relative_import,-15.412454
36,11.263705,-14.33197,190.553785,61.280219,-13.079721,-3.2028,Hong Kong,relative_export,-14.33197
38,9.99751,-7.968743,33.288427,1.159007,-13.564164,-10.650569,Singapore,relative_export,-13.564164


In [16]:
countries_affected=overall_df[overall_df["category"]=="relative_import"].head(10)["country"].tolist()
countries_affected

['Hong Kong',
 'Saudi Arabia',
 'China',
 'Israel',
 'Switzerland',
 'Netherlands',
 'Canada',
 'United Kingdom',
 'Ireland',
 'Germany']

In [17]:
country_to_index={'Hong Kong':"^HSI",
 'Saudi Arabia':"^TASI.SR",
 'China':"000001.SS",
 'Switzerland':"SMIN.SW",
 'Israel':"TA35.TA",
 'Mexico':"^MXX",
 'Malaysia':"^KLSE",
 'Vietnam':"^VNINDEX.VN",
 'Ireland':"^ISEQ",
 'Taiwan':"^TWII",
 "USA":"^GSPC"}

In [18]:
for country in country_to_index.keys():
    print(country)
    chartdata = yfinance.download (tickers = country_to_index[country], start = "2024-11-04", 
                                  interval = "1d")
    min=chartdata["Close"].min()
    max=chartdata["Close"].max()
    chartdata.to_csv(f"{country}_chart.csv")

Hong Kong
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Saudi Arabia


[*********************100%***********************]  1 of 1 completed


China


...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^VNINDEX.VN']: YFInvalidPeriodError("^VNINDEX.VN: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


Ireland


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Taiwan
USA


[*********************100%***********************]  1 of 1 completed


In [19]:
chartdata = yfinance.download (tickers = "^TASI.SR", start = "2025-01-01", 
                              interval = "1d")
chartdata

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR
Date,,,,,
2025-01-02,12102.549805,12120.490234,12069.709961,12087.940430,287054100
2025-01-06,12104.690430,12129.019531,12069.490234,12076.440430,401477400
2025-01-07,12113.290039,12161.910156,12091.860352,12112.120117,384440800
2025-01-08,12088.740234,12143.250000,12076.509766,12107.639648,535556600
2025-01-09,12097.750000,12121.419922,12041.349609,12094.049805,358742300
...,...,...,...,...,...
2025-10-02,11495.719727,11557.759766,11480.620117,11540.679688,322136300
2025-10-06,11605.200195,11645.139648,11533.709961,11534.509766,307744600


In [20]:
chartdata=chartdata.reset_index()

In [21]:
index_to_shortform_dict={"Switzerland":"switzerland-20","Vietnam":"vn"}
now = datetime.now()
formatted_date = now.strftime("%m/%d/%Y")
index="Switzerland"
index_shortform=index_to_shortform_dict[index]


url=f"https://www.investing.com/indices/{index_shortform}-historical-data#:~:text=Daily-,Download,-{formatted_date}"
    
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the first table on the page
    table = soup.find('table', class_="freeze-column-w-1 w-full overflow-x-auto text-xs leading-4")

    # Extract rows from the table
    rows = table.find_all('tr')

    # Extract data from each row
    table_data = []
    for row in rows:
        cells = row.find_all(['td', 'th'])  # Get all cells (both <td> and <th>)
        cell_data = [cell.get_text(strip=True) for cell in cells]  # Extract text
        table_data.append(cell_data)

    # Return the extracted table data
    print(table_data)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

[['', '', '', '', '', '', ''], ['Oct 09, 2025', '12,646.34', '12,627.34', '12,660.34', '12,613.84', '6.32M', '-0.05%'], ['Oct 08, 2025', '12,652.38', '12,551.83', '12,666.08', '12,551.76', '22.11M', '+1.04%'], ['Oct 07, 2025', '12,521.79', '12,545.14', '12,580.30', '12,509.79', '15.96M', '-0.24%'], ['Oct 06, 2025', '12,551.36', '12,498.73', '12,564.60', '12,471.45', '17.63M', '+0.35%'], ['Oct 03, 2025', '12,507.17', '12,450.07', '12,549.06', '12,440.74', '19.59M', '+0.64%'], ['Oct 02, 2025', '12,427.18', '12,446.94', '12,466.28', '12,395.52', '21.69M', '+0.54%'], ['Oct 01, 2025', '12,360.13', '12,193.90', '12,360.13', '12,189.42', '25.44M', '+2.07%'], ['Sep 30, 2025', '12,109.42', '12,005.98', '12,119.63', '11,985.61', '23.49M', '+0.86%'], ['Sep 29, 2025', '12,006.71', '11,968.52', '12,027.69', '11,958.35', '18.66M', '+0.64%'], ['Sep 26, 2025', '11,929.80', '11,869.07', '11,958.24', '11,867.89', '17.55M', '+0.45%'], ['Sep 25, 2025', '11,875.80', '11,934.43', '11,952.20', '11,875.80', '

In [22]:
pd.DataFrame(table_data[2:], columns=table_data[1])

,"Oct 09, 2025","12,646.34","12,627.34","12,660.34","12,613.84",6.32M,-0.05%
0,"Oct 08, 2025","12,652.38","12,551.83","12,666.08","12,551.76",22.11M,+1.04%
1,"Oct 07, 2025","12,521.79","12,545.14","12,580.30","12,509.79",15.96M,-0.24%
2,"Oct 06, 2025","12,551.36","12,498.73","12,564.60","12,471.45",17.63M,+0.35%
3,"Oct 03, 2025","12,507.17","12,450.07","12,549.06","12,440.74",19.59M,+0.64%
4,"Oct 02, 2025","12,427.18","12,446.94","12,466.28","12,395.52",21.69M,+0.54%
5,"Oct 01, 2025","12,360.13","12,193.90","12,360.13","12,189.42",25.44M,+2.07%
6,"Sep 30, 2025","12,109.42","12,005.98","12,119.63","11,985.61",23.49M,+0.86%
7,"Sep 29, 2025","12,006.71","11,968.52","12,027.69","11,958.35",18.66M,+0.64%
8,"Sep 26, 2025","11,929.80","11,869.07","11,958.24","11,867.89",17.55M,+0.45%
9,"Sep 25, 2025","11,875.80","11,934.43","11,952.20","11,875.80",23.23M,-0.86%


In [23]:
chartdata[chartdata["Date"]=="2025-07-21"]["Close"].values[0]

array([10981.16992188])

In [24]:
chartdata["Close"].max()

Ticker
^TASI.SR    12471.719727
dtype: float64

In [25]:
df=pd.DataFrame()

In [26]:
overall_df[overall_df["category"]=="relative_import"].dropna().tail(13)[["country","Januar","Februar","März","April","Mai","min"]].to_csv("relative_import_to_us_winners.csv", index=False)

KeyError: "['Januar'] not in index"

In [ ]:
overall_df[overall_df["category"]=="relative_import"].dropna().tail(13)[["country","Januar","Februar","März","April","Mai","min"]]

date,country,Januar,Februar,März,April,Mai,min
19,Italy,7.194198,31.418036,26.91497,7.198705,5.364668,5.364668
43,Australia,267.096727,255.796133,262.999442,32.132728,8.188337,8.188337
1,All other countries,8.658948,22.53502,27.170855,13.711973,9.668469,8.658948
17,India,9.024842,41.719909,65.377491,38.726445,28.679046,9.024842
3,Brazil,9.171763,38.117221,39.167065,13.195269,39.003309,9.171763
33,South/Central America,9.342285,22.593961,29.01242,25.323949,23.889427,9.342285
13,France,10.139636,10.779016,72.067419,30.348514,11.572642,10.139636
25,Mexico,13.32073,24.54988,24.34174,10.774312,19.25875,10.774312
51,Malaysia,16.46753,18.5046,21.652873,12.792348,14.049433,12.792348
11,European Union,21.181447,35.524398,80.823197,13.781978,22.432136,13.781978


In [27]:
overall_df[overall_df["category"]=="relative_import"].dropna().tail(10)["country"].tolist()

['Australia',
 'Brazil',
 'All other countries',
 'Mexico',
 'Malaysia',
 'South/Central America',
 'CAFTA-DR',
 'India',
 'Vietnam',
 'Taiwan']

In [ ]:
overall_df[overall_df["category"]=="relative_import"].head(10)[["country","Januar","Februar","März","April","Mai","min"]].to_csv("relative_import_to_us.csv", index=False)


KeyError: "['Januar'] not in index"

In [23]:
overall_df[overall_df["category"]=="relative_export"].head(10)[["country","Januar","Februar","März","April","Mai","min"]].to_csv("relative_export_to_us.csv", index=False)




KeyError: "['Januar'] not in index"

In [24]:
overall_df[overall_df["category"]=="relative_export"].head(10)[["country","Januar","Februar","März","April","Mai","min"]]

date,country,Januar,Februar,März,April,Mai,min
8,China,-19.168799,-8.231457,-14.660652,-31.846129,-42.624377,-42.624377
54,Switzerland,-35.643793,-5.085691,94.749153,328.28898,204.858901,-35.643793
44,Belgium,-20.474855,-6.753882,-3.078241,2.988985,-8.513479,-20.474855
36,Hong Kong,-7.416175,11.263705,-14.33197,190.553785,61.280219,-14.33197
46,Ireland,-9.317837,-0.233927,4.723924,7.050445,20.005119,-9.317837
38,Singapore,-8.18408,9.99751,-7.968743,33.288427,1.159007,-8.18408
6,Canada,1.176521,10.418587,6.249624,-5.156551,-7.324322,-7.324322
34,United Kingdom,-7.319118,60.431727,39.273619,47.567993,28.884202,-7.319118
22,"Korea, South",0.198236,7.970998,-4.180283,5.902951,-6.011876,-6.011876
30,Saudi Arabia,-5.758917,28.596763,5.508074,48.909667,25.624776,-5.758917
